# Basic tutorial: image data
#### Author: Matteo Caorsi

This short tutorial provides you with the basic functioning of *giotto-deep* API.

The main steps of the tutorial are the following:
 1. creation of a dataset
 2. creation of a model
 3. define metrics and losses
 4. run benchmarks
 5. visualise results interactively

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

import torch
from torch import nn

from gdeep.models import FFNet

from gdeep.visualisation import  persistence_diagrams_of_activations

from torch.utils.tensorboard import SummaryWriter
from gdeep.data import TorchDataLoader


from gtda.diagrams import BettiCurve

from gtda.plotting import plot_betti_surfaces

import optuna

# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [21]:
writer = SummaryWriter()

# Create your dataset

In [3]:
from torch.utils.data.sampler import SubsetRandomSampler

dl = TorchDataLoader(name="CIFAR10")
train_indices = []
for i in range(10240):
    train_indices.append(i)

print(len(train_indices))
dl_tr, dl_temp = dl.build_dataloader(batch_size=512, sampler=SubsetRandomSampler(train_indices))

print(len(dl_tr))

test_indices = []
for i in range(3072):
    test_indices.append(i)

dl_ts, dl_temp = dl.build_dataloader(batch_size=512, sampler=SubsetRandomSampler(test_indices))

dl_val = dl_ts

print(len(dl_ts))

10240
Files already downloaded and verified
Files already downloaded and verified
20
Files already downloaded and verified
Files already downloaded and verified
6


## Define and train your model

In [4]:
import torchvision.models as models
from gdeep.pipeline import Pipeline

model = nn.Sequential(models.resnet18(pretrained=True), nn.Linear(1000,10))

In [6]:
from torch.optim import SGD, Adam, RMSprop
from gdeep.search import gridsearch

# print(model)
loss_fn = nn.CrossEntropyLoss()

# pipe = Pipeline(model, (dl_tr, dl_ts), loss_fn, writer)
# pipe = Pipeline(model, (dl_tr, dl_ts), loss_fn, writer, True, "loss", 5)
pipe = Pipeline(model, [dl_tr, dl_val, dl_ts], loss_fn, writer)
# pipe = Pipeline(model, [dl_tr, dl_ts], loss_fn, writer)

# search = gridsearch.Gridsearch(pipe, "loss", 5)
# search.search([SGD, Adam], 1, lr=[0.001, 0.01])

# train the model
pipe.train(SGD, 2, cross_validation = True, batch_size = 512, lr=0.01)
# pipe.train([SGD, Adam], 1, lr=[0.001, 0.01])
# pipe.train([SGD, Adam], 1, lr=0.01)



TOTAL EPOCHS  2
Epoch 1
-------------------------------


d:\anaconda3\envs\giotto-deep2\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training loss: 1.585377  [20/20]
Time taken for this epoch: 7s
Validation results: 
 Accuracy: 3.3%,                 Avg loss: 0.000161 

Epoch 2
-------------------------------
Training loss: 1.291306  [20/20]
Time taken for this epoch: 5s
Validation results: 
 Accuracy: 4.0%,                 Avg loss: 0.000122 

Test results: 
 Accuracy: 4.0%,                 Avg loss: 0.000122 

Done!


# Gridsearch

In [12]:
from gdeep.search.gridsearch import Gridsearch
from torch.optim import SGD, Adam, RMSprop

loss_fn = nn.CrossEntropyLoss()

pipe = Pipeline(model, [dl_tr, dl_val, dl_ts], loss_fn, writer)
# pipe = Pipeline([model1, model2, ...], [[dl_tr, dl_val, dl_ts], [dl_tr2, dl_val2, dl_ts2], ...], loss_fn, writer)

search = Gridsearch(pipe, "loss", 10)
search.start([SGD, Adam], 1, lr=[0.001, 0.01])

[I 2021-07-24 14:13:41,543] A new study created in memory with name: no-name-2b6f1334-f317-41ff-8917-24a73a8db370


Epoch 1
-------------------------------
Training loss: 0.857715  [20/20]
Time taken for this epoch: 5s


[I 2021-07-24 14:13:47,424] Trial 0 finished with value: 0.8577152490615845 and parameters: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.0012334050172083419}. Best is trial 0 with value: 0.8577152490615845.


Validation results: 
 Accuracy: 4.3%,                 Avg loss: 0.000128 

Done!
Epoch 1
-------------------------------
Training loss: 0.892298  [20/20]
Time taken for this epoch: 5s


[I 2021-07-24 14:13:53,290] Trial 1 finished with value: 0.8922978043556213 and parameters: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.004655486428508404}. Best is trial 0 with value: 0.8577152490615845.


...

Validation results: 
 Accuracy: 4.1%,                 Avg loss: 0.000125 

Epoch 1
-------------------------------
Training loss: 0.915362  [20/20]
Time taken for this epoch: 5s


[I 2021-07-24 14:14:34,583] Trial 8 pruned. 


Validation results: 
 Accuracy: 4.2%,                 Avg loss: 0.000115 

Epoch 1
-------------------------------
Training loss: 0.893122  [20/20]
Time taken for this epoch: 5s


[I 2021-07-24 14:14:40,469] Trial 9 pruned. 


Validation results: 
 Accuracy: 4.2%,                 Avg loss: 0.000111 

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  4
  Number of complete trials:  6
Best trial:
  Value:  0.7816447019577026
  Params: 
    optimizer: <class 'torch.optim.sgd.SGD'>
    lr: 0.006913700491819268


# Gridsearch with multiple pipelines (models/datasets)

In [27]:
from gdeep.search.gridsearch import Gridsearch

pipe1 = Pipeline(model1, [dl_tr, dl_ts, dl_ts], loss_fn, writer)
pipe2 = Pipeline(model2, [dl_tr, dl_ts, dl_ts], loss_fn, writer)

search1 = Gridsearch(pipe1, "loss", 2)
search1.search([SGD, Adam], 1, lr=[0.001, 0.01])

search2 = Gridsearch(pipe1, "loss", 2)
search2.search([SGD, Adam], 1, lr=[0.001, 0.01])

NameError: name 'model1' is not defined

# Benchmarking a single model on multiple datasets

## Preparing multiple datasets

In [13]:
dataloaders_dicts = []
dl = TorchDataLoader(name="CIFAR10")
train_indices = []
for i in range(5120):
    train_indices.append(i)

dl_tr, dl_temp = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(train_indices))

test_indices = []
for i in range(1024):
    test_indices.append(i)

dl_ts, dl_temp = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(test_indices))

temp_dict = {}
temp_dict["name"] = "CIFAR10_5000"
temp_dict["dataloaders"] = (dl_tr, dl_ts)

dataloaders_dicts.append(temp_dict)


train_indices = []
for i in range(10240):
    train_indices.append(i)

dl_tr, dl_temp = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(train_indices))

test_indices = []
for i in range(2048):
    test_indices.append(i)

dl_ts, dl_temp = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(test_indices))

temp_dict = {}
temp_dict["name"] = "CIFAR10_10000"
temp_dict["dataloaders"] = (dl_tr, dl_ts)

dataloaders_dicts.append(temp_dict)

print(dataloaders_dicts[1]["name"])


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
CIFAR10_10000


## Benchmarking model

In [ ]:
from gdeep.pipeline import benchmark

bench = benchmark.Benchmark(writer)

bench.benchmark_model(model, dataloaders_dicts, loss_fn, SGD, 1, 0.01)  

# Benchmarking a single dataset on multiple models

## Preparing multiple models

In [14]:
models_dicts = []

model = nn.Sequential(models.resnet18(pretrained=True), nn.Linear(1000,10))
temp_dict = {}
temp_dict["name"] = "resnet18"
temp_dict["model"] = model

models_dicts.append(temp_dict)

model = nn.Sequential(models.vgg16(pretrained=True), nn.Linear(1000,10))
temp_dict = {}
temp_dict["name"] = "vgg16"
temp_dict["model"] = model

models_dicts.append(temp_dict)

## Benchmarking data

In [ ]:
bench = benchmark.Benchmark(writer)

bench.benchmark_data(model_dicts, (dl_tr, dl_ts), loss_fn, SGD, 1, 0.001)

## Benchmarking both

In [ ]:
from gdeep.search.benchmark import Benchmark
from torch.optim import SGD, Adam, RMSprop

loss_fn = nn.CrossEntropyLoss()

bench = Benchmark(writer)

bench.benchmark(models_dicts, dataloaders_dicts, loss_fn, optimizer = SGD, epochs = 1, learning_rate = 0.01, batch_size = 1024)

## Benchmarking + Gridsearch

In [46]:
from gdeep.search.benchmark import Benchmark
from gdeep.search.gridsearch import Gridsearch
from torch.optim import SGD, Adam, RMSprop

loss_fn = nn.CrossEntropyLoss()

bench = Benchmark(models_dicts, dataloaders_dicts, loss_fn, writer)

search = Gridsearch(bench, "loss", 2)
search.start([SGD, Adam], 10, 512, lr=[0.001, 0.01])

# bench.benchmark(optimizer = [SGD,Adam], epochs = 1, learning_rate = [0.001,0.01], batch_size = 1024)

[I 2021-07-24 15:42:56,194] A new study created in memory with name: no-name-d2768e24-3448-42e6-a904-043015ebff0d


****************************************
Performing Gridsearch on Dataset: CIFAR10_5000, Model: resnet18
Epoch 1
-------------------------------
Training loss: 1.662961  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000034 

Epoch 2
-------------------------------
Training loss: 1.632034  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000034 

Epoch 3
-------------------------------
Training loss: 1.599710  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000033 

Epoch 4
-------------------------------
Training loss: 1.571644  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000033 

Epoch 5
-------------------------------
Training loss: 1.598091  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000033 

Epoch

[I 2021-07-24 15:43:08,520] Trial 0 finished with value: 1.6274811029434204 and parameters: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.001095467238703449}. Best is trial 0 with value: 1.6274811029434204.


Training loss: 1.627481  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000033 

Done!
Epoch 1
-------------------------------
Training loss: 1.559724  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000033 

Epoch 2
-------------------------------
Training loss: 1.577237  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000033 

Epoch 3
-------------------------------
Training loss: 1.579534  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000033 

Epoch 4
-------------------------------
Training loss: 1.573544  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,                 Avg loss: 0.000033 

Epoch 5
-------------------------------
Training loss: 1.575348  [ 4/ 4]
Time taken for this epoch: 1s
Validation results: 
 Accuracy: 0.4%,    

[I 2021-07-24 15:43:20,765] Trial 1 finished with value: 1.5839738845825195 and parameters: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.0017754543788838376}. Best is trial 1 with value: 1.5839738845825195.
[I 2021-07-24 15:43:20,765] A new study created in memory with name: no-name-1ce0d042-6dcc-4d1c-bf25-4d6390963e20


...

Validation results: 
 Accuracy: 1.0%,                 Avg loss: 0.000065 

Done!
Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  1.1768337488174438
  Params: 
    optimizer: <class 'torch.optim.adam.Adam'>
    lr: 0.0019087573872860533
****************************************
Performing Gridsearch on Dataset: CIFAR10_10000, Model: vgg16
Epoch 1
-------------------------------
Training loss: 2.309373  [ 8/ 8]
Time taken for this epoch: 4s
Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Epoch 2
-------------------------------
Training loss: 2.302215  [ 8/ 8]
Time taken for this epoch: 4s
Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Epoch 3
-------------------------------
Training loss: 2.302715  [ 8/ 8]
Time taken for this epoch: 4s
Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Epoch 4
-------------------------------


[I 2021-07-24 15:45:30,252] Trial 0 finished with value: 2.3007373809814453 and parameters: {'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.004386187859870586}. Best is trial 0 with value: 2.3007373809814453.


Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Done!
Epoch 1
-------------------------------
Training loss: 2.304669  [ 8/ 8]
Time taken for this epoch: 4s
Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Epoch 2
-------------------------------
Training loss: 2.303228  [ 8/ 8]
Time taken for this epoch: 4s
Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Epoch 3
-------------------------------
Training loss: 2.305750  [ 8/ 8]
Time taken for this epoch: 4s
Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Epoch 4
-------------------------------
Training loss: 2.297784  [ 8/ 8]
Time taken for this epoch: 4s
Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Epoch 5
-------------------------------
Training loss: 2.302551  [ 8/ 8]
Time taken for this epoch: 4s
Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Epoch 6
---------------------

[I 2021-07-24 15:46:11,493] Trial 1 finished with value: 2.3025522232055664 and parameters: {'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.0031264922643608408}. Best is trial 0 with value: 2.3007373809814453.


Validation results: 
 Accuracy: 0.2%,                 Avg loss: 0.000092 

Done!
Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  2.3007373809814453
  Params: 
    optimizer: <class 'torch.optim.adam.Adam'>
    lr: 0.004386187859870586


# Simply use interpretability tools

In [ ]:
from gdeep.analysis.interpretability import Interpreter

inter = Interpreter(model, method="IntegratedGradients")
inter.interpret_image(next(iter(dl_tr))[0][0].reshape(1,3,32,32), 6);


# Extract inner data from your models

In [ ]:
from gdeep.models import ModelExtractor

me = ModelExtractor(model, loss_fn)

lista = me.get_layers_param()

for k, item in lista.items():
    print(k,item.shape)


In [ ]:
x = next(iter(dl_tr))[0][0]
if x.dtype is not torch.int64:
    res = me.get_decision_boundary(x, n_epochs=1)
    res.shape

In [ ]:
x = next(iter(dl_tr))[0]
list_activations = me.get_activations(x)
len(list_activations)


In [ ]:
x, target = next(iter(dl_tr))
if x.dtype is torch.float:
    for gradient in me.get_gradients(x, target=target)[1]:
        print(gradient.shape)

# Visualise activations and other topological aspects of your model

In [ ]:
from gdeep.visualisation import Visualiser

vs = Visualiser(pipe)

vs.plot_data_model()
vs.plot_activations(x)
vs.plot_persistence_diagrams(x)


In [ ]:
vs.plot_interpreter_image(inter);